In [1]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import lstm, time #helper libraries
%load_ext autoreload
%autoreload 2

/home/reut/virtualenv/py-keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path

In [3]:
working_csv = 'hist2_out.csv'
original_csv = 'IWM-history2.csv'

In [4]:
# create formated csv file if it does not exist
if not os.path.isfile(working_csv):
    df = pd.read_csv(original_csv)
    close = df['Adj Close']
    close.to_csv(working_csv, index=False)

In [5]:
def stock_lstm(window_size=50, predict_len=50, lstm_par1=50, lstm_par2=100, epoch=15, model=None, X_train=None, y_train=None, X_test=None, y_test=None):

    # create a new model if one wasn't provided
    if model==None:
        X_train, y_train, X_test, y_test = lstm.load_data(working_csv, window_size, True)
        model = Sequential()

        model.add(LSTM(
            input_dim=1,
            output_dim=lstm_par1,
            return_sequences=True))
        model.add(Dropout(0.2))

        model.add(LSTM(
            lstm_par2,
            return_sequences=False))
        model.add(Dropout(0.2))

        model.add(Dense(
            output_dim=1))
        model.add(Activation('linear'))

        start = time.time()
        model.compile(loss='mse', optimizer='rmsprop')
        print('compilation time : ', time.time() - start)
    
    #Step 3 Train the model
    model.fit(
        X_train,
        y_train,
        batch_size=512,
        nb_epoch=epoch,
        validation_split=0.05)
    
    #Step 4 - Plot the predictions!
    predictions = lstm.predict_sequences_multiple(model, X_test, window_size, predict_len)
    lstm.plot_results_multiple(predictions, y_test, predict_len)
    
    cache = (X_train, y_train, X_test, y_test, model, predictions)
    return cache

In [ ]:
cache1 = stock_lstm(window_size=50, predict_len=50, lstm_par1=100, lstm_par2=100, epoch=3)

compilation time :  0.02648758888244629
Train on 3808 samples, validate on 201 samples
Epoch 1/3
3808/3808 [==============================] - 7s 2ms/step - loss: 1.1817 - val_loss: 0.7489
Epoch 2/3
